# 第9章: RNN, CNN

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Jul  7 03:16:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install stemming

  Created wheel for stemming: filename=stemming-1.0.1-cp36-none-any.whl size=11139 sha256=5cc968535c06561243a12d8e48a97652d9a317f651cd3f7e5cbbc0b897298ff7
  Stored in directory: /root/.cache/pip/wheels/e8/05/2e/2ddeb64d4464b854b48323f9676528c17560da7d153db7b0e2
Successfully built stemming


In [ ]:
import spacy
from stemming.porter2 import stem
from collections import Counter
import numpy as np
from tqdm import tqdm
import pickle
import random

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils.rnn as rnn
from torch.nn.utils.rnn import pad_sequence

## 80. ID番号への変換
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
with open("./drive/My Drive/Colab Notebooks/data/NewsAggregatorDataset/newsCorpora.csv", "r") as intxt:
    inline = intxt.readlines()
    
select_publisher = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
datas = []

for line in inline:
    if line != "":
        ll = line.split("\t")
        if ll[3] in select_publisher:
            datas.append((ll[4],ll[1]))

In [ ]:
def get_feat_from_sentence(string):
    sl = nlp.make_doc(string)
    string = [i.lemma_.lower() for i in sl]
    return string

def data2stemmed(data):
    return [(label, get_feat_from_sentence(string)) for label,string in data]

def get_feature_stems(stemmed):
    counter = Counter([tok for _,toks in stemmed for tok in toks])
    return [stem for stem,num in counter.most_common() if 2<=num]

def make_feature_dic(features):
    dic = {}
    for i, word in enumerate(features):
        dic[word] = i+1
    return dic

def get_id(stemmed, feature_dic):
    if stemmed in feature_dic:
        return feature_dic[stemmed]
    else:
        return 0

def get_ids(stemmedl, feature_dic):
    return [get_id(i, feature_dic) for i in stemmedl]

def stemmed2ids(stemmed, feature_dic):
    return [(label, get_ids(steml, feature_dic)) for label,steml in stemmed]

In [ ]:
stemmed = data2stemmed(datas)
features = get_feature_stems(stemmed)
feature_dic = make_feature_dic(features)
ids = stemmed2ids(stemmed, feature_dic)

In [ ]:
len(features)

8600

In [ ]:
print(get_id("-", feature_dic))
print(get_id("hogehoge", feature_dic))

1
0


In [ ]:
stemmed[0][1]

['europe', 'reach', 'crunch', 'point', 'on', 'bank', 'union']

In [ ]:
get_ids(stemmed[0][1], feature_dic)

[247, 858, 0, 927, 13, 60, 957]

In [ ]:
for i in ids[:3]:
   print(i)

for i in ids[-3:]:
   print(i)

('b', [247, 858, 0, 927, 13, 60, 957])
('b', [54, 604, 1, 2586, 43, 0, 61, 54, 5, 1249, 3, 419, 72, 45, 12, 7, 6])
('b', [43, 0, 0, 9, 6314, 116, 550, 4, 1900, 243, 4220])
('m', [14, 0, 1611, 116, 8600, 4, 669, 1737, 1, 126])
('m', [246, 1301, 49, 0, 4943, 4, 126, 65, 8, 4975, 158])
('m', [337, 707, 1692, 745, 3, 246, 1, 104, 2123, 2124, 20, 2898, 11, 464, 7, 6])


In [ ]:
def ids2ids_onehot(ids, features):
    size = len(features) + 1
    identity = np.identity(size)
    return [(label, identity[i]) for label,i in tqdm(ids)]

In [ ]:
#ids_onehot = ids2ids_onehot(ids, features)

In [ ]:
#for i in ids_onehot[:3]:
#   print(i)

#for i in ids_onehot[-3:]:
#   print(i)

## 81. RNNによる予測

In [ ]:
#random.shuffle(ids_onehot)
random.shuffle(ids)

In [ ]:
class Model81(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, out_dim):
        super(Model81, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers=1, batch_first=True)
        self.out = nn.Linear(hidden_dim, out_dim)
        nn.init.normal_(self.embed.weight, 0.0, 1.0)
        #self.loss = nn.CrossEntropyLoss()
    
    def predict(self, x, h=None):
        x = self.embed(x.to(torch.int64))
        x, hp = self.rnn(x, h)
        h = self.out(hp)
        #print(h)
        h = h.squeeze(0)
        return h

    def forward(self, x):
        label = self.predict(x, None)
        #return self.loss(label, ans)
        return label

In [ ]:
model = Model81(8601, 300, 50, 4)

In [ ]:
print(model.predict(torch.tensor(ids[0][1]).unsqueeze(0)))
print(ids[0][0])

tensor([[-0.0944, -0.3514, -0.1393, -0.9557]], grad_fn=<SqueezeBackward1>)
e


In [ ]:
print(model.predict(torch.tensor(ids[1][1]).unsqueeze(0)))
print(ids[1][0])

tensor([[-0.3029, -0.3453, -0.9609,  0.1288]], grad_fn=<SqueezeBackward1>)
t


## 82. 確率的勾配降下法による学習

In [ ]:
label_dic = {"b":0, "e":1, "m":2, "t":3}

def div_vecs(vecs):
    x = [np.array(i[1]) for i in vecs]
    label = [label_dic[i[0]] for i in vecs]
    return x,label

trains = ids[:len(ids)*4//5]
print("train data :",len(trains))

tests = ids[len(ids)*8//10:len(ids)*9//10]
print("valid data :",len(tests))

valids = ids[len(ids)*9//10:]
print("test data :",len(valids))

train data : 10684
valid data : 1336
test data : 1336


In [ ]:
trainx, trainy = div_vecs(trains)
testx, testy = div_vecs(tests)
validx, validy = div_vecs(valids)

In [ ]:
datasets = [trainx, trainy, testx, testy, validx, validy]

In [ ]:
def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

In [ ]:
class Model81(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, out_dim):
        super(Model81, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers=1, batch_first=True)
        self.out = nn.Linear(hidden_dim, out_dim)
        nn.init.normal_(self.embed.weight, 0.0, 1.0)
    
    def predict(self, x, h=None):
        x = self.forward(x, h)
        x = F.softmax(x, dim=1)
        return x

    def forward(self, x, h=None):
        x = self.embed(x.to(torch.int64))
        x, hp = self.rnn(x, h)
        h = self.out(hp)
        #print(h)
        h = h.squeeze(0)
        return h

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, status, use_gpu=False):
        dic = {"train":0, "test":1, "valid":2}
        self.status = status
        vecs = dataset
        self.data = vecs[dic[status]*2]
        self.label = vecs[dic[status]*2+1]
        self.data_num = len(self.data)
        self.use_gpu = use_gpu
    
    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label =  self.label[idx]
        return out_data, out_label
    
    def collate(self, batch):
        datas, labels = list(zip(*batch))
        datas = [torch.tensor(data) for data in datas]
        datas = pad_sequence(datas, batch_first=True)
        if self.use_gpu:
            return try_gpu(datas), try_gpu(torch.tensor(labels))
        else:
            return datas, torch.tensor(labels)

def generator(dataset, status, batch_size, shuffle=True, use_gpu=False):
    data_set = Dataset(dataset, status, use_gpu)
    return torch.utils.data.DataLoader(data_set, collate_fn = data_set.collate, batch_size=batch_size, shuffle=shuffle)

In [ ]:
ds = generator(datasets, "train", 1)
for x, y in ds:
  print(x,y)
  break

tensor([[ 699, 1041,    3,  521,   15, 8050, 4370, 5606, 4414,   20, 1827,  521,
         1414,    7,    6]]) tensor([3])


In [ ]:
ds = generator(datasets, "train", 100)
for x, y in ds:
  print(x,y)
  break

tensor([[  14,    0, 5257,  ...,    0,    0,    0],
        [ 340,  797, 1296,  ...,    0,    0,    0],
        [  17,   33,    1,  ...,    0,    0,    0],
        ...,
        [2057,   11,  926,  ...,    0,    0,    0],
        [3953,   11,  861,  ...,    0,    0,    0],
        [1175, 4044, 1262,  ...,    0,    0,    0]]) tensor([0, 1, 0, 3, 0, 1, 2, 0, 0, 3, 0, 1, 1, 2, 1, 1, 1, 0, 2, 1, 3, 0, 1, 1,
        0, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        0, 2, 3, 1, 3, 0, 0, 1, 0, 0, 1, 1, 0, 0, 3, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 2, 0, 1, 3, 0, 2, 3, 1, 3, 0, 0, 0, 1, 2, 1, 0, 1, 3, 1, 3, 1, 0, 1,
        1, 1, 0, 1])


In [ ]:
def accuracy(dataset, md, use_gpu=False):
    with torch.no_grad():
        if use_gpu:
            predicts = [md.predict(try_gpu(torch.tensor(dat).unsqueeze(0))) for dat in dataset.data]
        else:
            predicts = [md.predict(torch.tensor(dat).unsqueeze(0)) for dat in dataset.data]
        return np.mean([int(p.argmax()==a) for p,a in zip(predicts, dataset.label)])

In [ ]:
## batch>=2のときスコアが明らかに悪い(パディングがヤバそう)　要改善
class TRAINER:
    def __init__(self, model, criterion, vecs, optimizer, gen_batchsize, max_iter, use_gpu=False):
        self.train_generator = generator(datasets, "train", gen_batchsize, True,  use_gpu)
        self.valid_generator = generator(datasets, "valid", 1, True, use_gpu)
        self.train_dataset = Dataset(datasets, "train")
        self.valid_dataset = Dataset(datasets, "valid")
        self.model = model
        if use_gpu:
            self.model = try_gpu(self.model)
        self.criterion = criterion
        self.optimizer = optimizer
        self.max_iter = max_iter
        self.use_gpu = use_gpu

    def train(self):
        self.model.train()
        train_losses = []
        for x,y in self.train_generator:
            out = self.model(x)
            loss = self.criterion(out, y)
            #self.optimizer.zero_grad()
            self.model.zero_grad()
            loss.backward()
            self.optimizer.step()
            train_losses.append(loss.item())
        return accuracy(self.train_dataset, model, self.use_gpu), np.mean(train_losses)
        
    def valid(self):
        self.model.eval()
        valid_losses = []
        for x, y in self.valid_generator:
            with torch.no_grad():
                out = self.model(x)
                loss = self.criterion(out, y)
                valid_losses.append(loss.item())
        return accuracy(self.valid_dataset, model, self.use_gpu), np.mean(valid_losses)

    def learning(self):
        for ep in range(self.max_iter):
            train_acc, train_loss = self.train()
            valid_acc, valid_loss = self.valid()
            print(train_acc, valid_acc, train_loss, valid_loss)

In [ ]:
model = Model81(8601, 300, 50, 4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
trainer = TRAINER(model, criterion, datasets, optimizer, 1, 5, False)

In [ ]:
train_dataset = Dataset(datasets, "train")
accuracy(train_dataset, model)

0.293803818794459

In [ ]:
print("train_accuracy / valid_accuracy / train_loss / valid_loss")
trainer.learning()

train_accuracy / valid_accuracy / train_loss / valid_loss
0.7308124298015725 0.7073353293413174 0.9822527083602446 0.8684948053445198
0.7687195806813928 0.7215568862275449 0.789449693042046 0.8144324706312448
0.7876263571695994 0.7395209580838323 0.6887827925964378 0.7597779967110397
0.8177648820666417 0.7402694610778443 0.622257382522957 0.7541777664369591
0.8306813927368027 0.7657185628742516 0.562488233715828 0.7783636722402542


## 83. ミニバッチ化・GPU上での学習

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
model = Model81(8601, 300, 50, 4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
trainer = TRAINER(model, criterion, datasets, optimizer, 4, 10, True)

In [ ]:
print("train_accuracy / valid_accuracy / train_loss / valid_loss")
trainer.learning()

train_accuracy / valid_accuracy / train_loss / valid_loss
0.5446461999251216 0.5486526946107785 1.1458693519932015 1.1041750812869586
0.5768438786971172 0.5688622754491018 1.1166642746622757 1.0642547464656258
0.6219580681392737 0.5988023952095808 1.0890664964613064 1.028139219357225
0.6846686634219393 0.6788922155688623 1.0135935698274994 0.934703773411805
0.7100336952452265 0.7013473053892215 0.9585364049336117 0.8930339098512056
0.7031074503931112 0.6893712574850299 1.0081992586959574 0.9004446455461537
0.5862036690378135 0.5845808383233533 0.9665696329389427 1.0785217452905849
0.662392362411082 0.6482035928143712 1.07155192914816 0.9705307362768465
0.7057281916885062 0.686377245508982 0.9830898343568346 0.9182277883508962
0.7141520029951329 0.6953592814371258 0.9444791389143186 0.9010971833667355


## 84. 単語ベクトルの導入

In [ ]:
import gensim
googlenews_w2v = gensim.models.KeyedVectors.load_word2vec_format('./drive/My Drive/Colab Notebooks/data/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
import copy
def init_emb_w2v(model):
    for i, token in enumerate(features):
        if token in googlenews_w2v:
            vc = copy.deepcopy(googlenews_w2v[token])
            model.embed.weight.data[i] = torch.tensor(vc)

In [ ]:
model = Model81(8601, 300, 50, 4)
init_emb_w2v(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
trainer = TRAINER(model, criterion, datasets, optimizer, 1, 5, False)

In [ ]:
print("train_accuracy / valid_accuracy / train_loss / valid_loss")
trainer.learning()

train_accuracy / valid_accuracy / train_loss / valid_loss
0.652096593036316 0.6317365269461078 1.0577447965569606 0.9893915286666887
0.7288468738300262 0.7148203592814372 0.9263852405404022 0.8395172570773517
0.7658180456757768 0.718562874251497 0.7914477826974092 0.8162118859427373
0.7821976787719955 0.7485029940119761 0.6981621248252222 0.8251152227174011
0.8110258330213403 0.7350299401197605 0.6243131635911742 0.7703471740921765


## 85. 双方向RNN・多層化

In [ ]:
class Model85(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, out_dim):
        super(Model85, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
        self.out = nn.Linear(hidden_dim*2, out_dim)
        nn.init.normal_(self.embed.weight, 0.0, 1.0)
    
    def predict(self, x, h=None):
        x = self.forward(x, h)
        x = F.softmax(x, dim=1)
        return x

    def forward(self, x, h=None):
        x = self.embed(x.to(torch.int64))
        x, hp = self.rnn(x, h)
        h = hp[-2:]
        h = h.transpose(0,1)
        h = h.contiguous().view(-1, h.size(1) * h.size(2))
        h = self.out(h)
        return h

In [ ]:
model = Model85(8601, 300, 50, 4)
init_emb_w2v(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
trainer = TRAINER(model, criterion, datasets, optimizer, 1, 5, False)

In [ ]:
print("train_accuracy / valid_accuracy / train_loss / valid_loss")
trainer.learning()

train_accuracy / valid_accuracy / train_loss / valid_loss
0.7704043429427181 0.7402694610778443 0.8718260271098233 0.745074547161167
0.8592287532759266 0.7859281437125748 0.6132917340548394 0.6117972818947945
0.8887120928491202 0.8023952095808383 0.4511631824753068 0.579840137231438
0.9239985024335455 0.8270958083832335 0.32984242661446256 0.5580287224444443
0.9541370273305878 0.8353293413173652 0.2435087069509945 0.5792550135639964


In [ ]:
model = Model85(8601, 300, 50, 4)
init_emb_w2v(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.02)
trainer = TRAINER(model, criterion, datasets, optimizer, 8, 10, True)

In [ ]:
print("train_accuracy / valid_accuracy / train_loss / valid_loss")
trainer.learning()

train_accuracy / valid_accuracy / train_loss / valid_loss
0.6099775365031823 0.592814371257485 0.9679292242550207 1.070125158229274
0.7585174092100337 0.7447604790419161 0.7881251931368948 0.7404168622936317
0.7746162485960314 0.7335329341317365 0.677808401001963 0.7321860723777446
0.8230063646574317 0.7747005988023952 0.5849296065984283 0.6266814275772986
0.7677836016473231 0.7170658682634731 0.5088952729913467 0.7930596894668248
0.8708348932983901 0.8023952095808383 0.4452917826456759 0.5594385755276252
0.8837514039685511 0.8046407185628742 0.3801768596289668 0.5803251109437314
0.9116435791838263 0.8151197604790419 0.3275278561419534 0.5589102379040803
0.9182890303257207 0.8143712574850299 0.27757048916383953 0.6037391513407587
0.9412205166604268 0.8173652694610778 0.23550863502669833 0.5867536088039061


## 86. 畳み込みニューラルネットワーク (CNN)

In [ ]:
class Model86(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, out_dim):
        super(Model86, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
        self.conv = nn.Conv1d(embed_dim, hidden_dim, kernel_size=3, padding=1)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=3)
        self.out = nn.Linear(hidden_dim, out_dim)
        nn.init.normal_(self.embed.weight, 0.0, 1.0)
    
    def predict(self, x):
        x = self.forward(x)
        x = F.softmax(x, dim=1)
        return x

    def forward(self, x):
        x = self.embed(x.to(torch.int64))
        x = self.conv(x.transpose(-1,-2))
        x = self.act(x)
        x = F.max_pool1d(x, x.size(-1))
        x = x.squeeze(-1)
        x = self.out(x)
        return x

## 87. 確率的勾配降下法によるCNNの学習

In [ ]:
model = Model86(8601, 300, 50, 4)

In [ ]:
print(model.predict(torch.tensor(ids[0][1]).unsqueeze(0)))

tensor([[0.0877, 0.2863, 0.3967, 0.2292]], grad_fn=<SoftmaxBackward>)


In [ ]:
model = Model86(8601, 300, 50, 4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
trainer = TRAINER(model, criterion, datasets, optimizer, 1, 5, False)

In [ ]:
print("train_accuracy / valid_accuracy / train_loss / valid_loss")
trainer.learning()

train_accuracy / valid_accuracy / train_loss / valid_loss
0.752807937102209 0.7155688622754491 1.457637474694676 3.0803044119107206
0.6767128416323475 0.6422155688622755 6.809485087754355 11.905877648207417
0.8400411830774991 0.7664670658682635 11.68659215040173 13.791731543565318
0.8862785473605391 0.7739520958083832 8.354457061628628 16.8977603638319
0.9288655934107076 0.8106287425149701 5.021623709647513 14.891164257820476
